# Inverted Pendulum

My goal for this project is to explore control systems for an inverted pendulum. The basis setup is we have a cart of mass $M$ and mass at the top of a pole on the care of mass $m$. We will assume the pole itself to be massless. 

Let $\theta$ represent the angle between vertical and the pole, let $v_1$ represent the velocity of the cart and $v_2$ the velocity of the pole mass. Let $F$ be the force exerted on the cart (which we constraint to move along the x-axis) and the pole will be of length $l$. 

<img src="inverted_pendulum_graphic.jpg" width="400"/>

### Calculate the Lagrangian

Let's start by calculating the kinematics of the system. By the [principal of least action](https://www.feynmanlectures.caltech.edu/II_19.html) the path of the pole mass and the car minimize the functional 

$$
J[x, \theta] = \int_{t_0}^{t_1} L 
$$

Where $L$ is the Lagrangian $L = T - U$ which represents the difference in kinetic and potential energy of the system. The position of the cart at time $t$ is $(x(t), 0)$ and the position of the pole mass is $(x + l \sin{\theta}, l\cos{\theta})$ we can express the kinetic energy of the system as: 

$$ 

\begin{aligned}
T &= \frac{1}{2}M v_1^2 + \frac{1}{2} m v_2^2 \\

T &= \frac{1}{2}(M) \dot{x}^2 + \frac{1}{2}m((\dot{x} + l \dot{\theta} \cos{\theta})^2 + (-l\dot{\theta} \sin{\theta})^2) \\

T &= \frac{1}{2} (m + M) \dot{x}^2 + \frac{1}{2} m l^2 \dot{\theta}^2 - m l \dot{\theta} \dot{x} \cos{theta}

\end{aligned}
$$

And the potential energy can be expressed simply: 

$$
\begin{aligned}
U &= m g l \cos{\theta} \\
\end{aligned}
$$

### Apply the principle of least action to calculate the differential equations
By applying [D'Alambert's principle](https://en.wikipedia.org/wiki/D%27Alembert%27s_principle#:~:text=D'Alembert's%20principle%20generalizes%20the,system%2C%20result%20in%20dynamic%20equilibrium.&text=D'Alembert's%20principle%20can%20be,constraints%20that%20depend%20on%20velocities.), the euler-lagrange equations now include the non-conservative force F. 


$$
\begin{aligned}
\frac{\partial L}{\partial x} - \frac{d}{dt} \frac{\partial L}{\partial \dot{x}} &= F \\

\frac{\partial L }{\partial \theta} -  \frac{d}{dt}  \frac{\partial L}{\partial \dot{\theta}} &= 0
\end{aligned}
$$

To review the derivation of the Euler-lagrange equations from the principle of least action I found a [good video](https://www.youtube.com/watch?v=sFqp2lCEvwM). Based on the equations for kinetic and potential energy above, we can now calculate all the relevant partial derivatives. 

$$
\begin{aligned}
\frac{\partial L}{\partial x} &= 0 \\

\frac{\partial L}{\partial \dot{x}} &= (M + m) \dot{x} + ml \dot{\theta} \cos{\theta} \\

\frac{\partial L}{\partial \theta} &=  - m l \dot{x} \theta \sin{\theta} + m g l \sin(\theta) = m l \sin{\theta} (g - \dot{x} \theta) \\

\frac{\partial L}{\partial \dot{\theta}} &= m l^2 \dot{\theta} + m l \dot{x} \cos{\theta} \\

\end{aligned}
$$

Now plugging those partials into the euler-lagrange equations we ultimately get: 

$$
\begin{aligned}
- (M + m) \ddot{x} - m l \ddot{\theta} \cos{\theta} + m l \dot{\theta}^2 \sin{\theta} &= F \\
g \sin{\theta} - l \ddot{\theta} - \ddot{x} \cos{\theta} &= 0

\end{aligned}
$$

We can manipulate the above equations to solve for $\ddot{\theta}$ and $\ddot{x}$. Doing that we get 

$$
\begin{aligned}
\ddot{x} &= \frac{-F - mg \cos{\theta} \sin{\theta} + ml \dot{\theta}^2 \sin{\theta}}{M + m \sin^2{\theta}} \\

\ddot{\theta} &= \frac{F \cos{\theta} - m l \theta^2 \sin{\theta} \cos{\theta} + (M + m) g \sin{\theta}}{Ml + ml \sin^2{\theta}}
\end{aligned}
$$

### Linearization of the problem
To simply the problem further, we make the assumption of small angles and therefore are able to linearize the above equations. 

$$
\begin{aligned}
-(M + m) \ddot{x} - ml \ddot{\theta} &= F \\
g \theta - l \ddot{\theta} - \ddot{x} = 0
\end{aligned}
$$

With minimal algebraic manipulation we can rewrite these equations as:

$$
\begin{aligned}
\ddot{x} &= - \frac{F}{M} - \frac{mg\theta}{M} \\

\ddot{\theta} &= \frac{mg}{Ml} \theta + \frac{g}{l} \theta + \frac{F}{Ml}
\end{aligned}
$$

We can now write everything out as four first order first-order differential equations. 

### Write in matrix form

We will define our state vector like so
$$
\mathbf{x} = \begin{bmatrix} x_1 \ x_2 \ x_3 \ x_4 \end{bmatrix} = \begin{bmatrix} x \ \dot{x} \ \theta \ \dot{\theta} \end{bmatrix} \\
$$

We will define our input vector as 
$$
\mathbf{u} = \begin{bmatrix} F \end{bmatrix}
$$

The first order system of differential equations can then be expressed in matrix form:
$$
\dot{\mathbf{x}} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}
$$

Where A is defined like so: 
$$
\mathbf{A} = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 0 & 0 & - \frac{mg}{M} & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & \frac{mg}{Ml} + \frac{g}{l} & 0 \end{bmatrix}
$$

and B is defined like so
$$
\mathbf{B} = \begin{bmatrix} 0 \ - \frac{1}{M} \ 0 \ \frac{1}{Ml} \end{bmatrix}
$$

### Write some code

Now that we have a mathmatical understanding of the problem, let's write out some functions to return the matrix $A$ and $B$ from above, a function to solve for $X$ and $U$ given a matrix with constants of proportional control $K$, and a couple functions to plot the state of the system over time. 

In [50]:
import numpy as np
import scipy

def get_matrices_A_B(M, m, g, l):
    """
    Returns the system matrix A and input matrix B.

    Parameters:
    M (float): Mass of the cart.
    m (float): Mass of the pendulum.
    g (float, optional): Acceleration due to gravity. 
    l (float, optional): Length of the pendulum.

    Returns:
    A (numpy.ndarray): System matrix.
    B (numpy.ndarray): Input matrix.
    """

    A = np.array([
        [0, 1, 0, 0],
        [0, 0, -(m * g) / M, 0],
        [0, 0, 0, 1],
        [0, 0, (m * g) / (M * l) + g / l, 0]
    ])

    B = np.array([
        [0],
        [-1 / M],
        [0],
        [1 / (M * l)]
    ])

    return A, B


In [89]:
def solve_cart_control(t, X0, A, B, K):
    """
    Given an initial condition, solve the control problem at each time step. 

    Parameters: 
    -----------
    t (numpy.ndarray): time values with shape (n+1, )
    X0 (numpy.ndarray): Initial conditions on state variables (4,1)
    A (numpy.ndarray): System matrix (4,4).
    B (numpy.ndarray): Input matrix (4,1).
    K (numpy.ndarray): Porportional control matrix (1, 4).

    Returns
    -------
    X (numpy.ndarray): the state vector at each time (n+1, 4). 
    U (numpy.ndarray): the control values (n+1, 1). 

    """

    # Porportional control. 
    def u(x):
        return K @ x

    # time independent differential equations.
    def dxdt(x, _):
        dxdt = A @ x + B @ u(x)
        return np.resize(dxdt, 4)

    X = scipy.integrate.odeint(dxdt, np.resize(X0, 4), t)
    U = K @ X.T

    return X, U 

In [109]:
import matplotlib.pyplot as plt 

from IPython.display import HTML
from matplotlib import animation

def plot_state_vector_vs_time(t, X):
    """
    Simple function to plot x, x_dot, theta, theta_dot against time. 
    """

    plt.plot(t, X[:,0], label="x")
    plt.plot(t, X[:,1], label="x dot")
    plt.plot(t, X[:,2], label="theta")
    plt.plot(t, X[:,3], label="theta dot")


    plt.title('State variables vs time')
    plt.legend()
    plt.xlabel("t [s]")

    plt.show()


def animate_cart_and_pendulum(t, X):
    """
    Animate the position of the cart and the inverse pendulum based on the passed in state matrix and time vector. 
    """

    fig = plt.figure(figsize=(8,6.4))
    ax = fig.add_subplot(111,autoscale_on=False,\
                        xlim=(-1.5,5),ylim=(-0.4,5))
    ax.set_xlabel('position')
    ax.get_yaxis().set_visible(False)

    crane_rail, = ax.plot([-1.5,5],[-0.2,-0.2],'k-',lw=4)

    start, = ax.plot([-1,-1],[-1.5, 5],'k:',lw=2)
    objective, = ax.plot([0,0],[-0.5, 5],'k:',lw=2)

    mass1, = ax.plot([],[],linestyle='None',marker='s',\
                    markersize=40,markeredgecolor='k',\
                    color='blue',markeredgewidth=2)
    mass2, = ax.plot([],[],linestyle='None',marker='o',\
                    markersize=20,markeredgecolor='k',\
                    color='blue',markeredgewidth=2)
    line, = ax.plot([],[],'o-',color='blue',lw=4,\
                    markersize=6,markeredgecolor='k',\
                    markerfacecolor='k')

    time_template = 'time = %.1f [s]'
    time_text = ax.text(0.7,0.9,'',transform=ax.transAxes)

    theta_template = 'theta = %.4f [rad]'
    theta_text = ax.text(0.7,0.86,'',transform=ax.transAxes)

    def init():
        mass1.set_data([],[])
        mass2.set_data([],[])
        line.set_data([],[])
        time_text.set_text('')
        theta_text.set_text('')
        return line, mass1, mass2, time_text, theta_text

    def animate(i):
        theta = X[i,2]
        x1, y1 = X[i,0], -0.1
        x2, y2 = x1 + l*np.sin(theta), l*np.cos(theta)
        mass1.set_data([x1],[y1])
        mass2.set_data([x2],[y2])
        line.set_data([x1,x2],[y1,y2])
                    
        time_text.set_text(time_template % t[i])
        theta_text.set_text(theta_template % theta)

        return line, mass1, mass2, time_text, theta_text

    ani = animation.FuncAnimation(fig, animate, \
            np.arange(len(t)), \
            blit=False,init_func=init)
    
    return ani

## Sanity check 
First, as a quick sanity check lets just simulate an inverted pendulum absent any external force. We will set initial conditions here that we will continue to use for the rest of our experiments. 

In [111]:
# Set initial conditions
M, m = 17, 2
l = 4
g = 9.8
q1, q2, q3, q4 = 1, 1, 1, 1
r = 10

# Get the linearized equations for the state vector and force vector.
A, B = get_matrices_A_B(M, m, g, l)

# Let's set the proportional control matrix equal to zero. 
K = np.zeros((1, 4))

X0 = np.array([-1, 0, 0.1, 0])
t = np.linspace(0, 10, 50)

X, U = solve_cart_control(t, X0, A, B, K)

%matplotlib notebook
HTML(animate_cart_and_pendulum(t, X).to_html5_video())

<IPython.core.display.Javascript object>

# Solve the control problem 
Alright, that sanity check looks reasonable. The pendulum accelerates downwards on the right hand side of the cart, forcing the card to slide to the left.

Now let's try to find constants for the proportional control matrix $K$ (where $U = K (X - X_{target})$ ) that lead to stability. 

## Apply a Linear Quadratic Regulator with infinite time horizons to solve the controls problem
One method we can use to solve this time-invarient linear system is Linear Quadratic Regulators (LQR) which we can use to find the optimal control input by minimizing the quadratic cost function: 

$$
\int_0^\infty (x^T Q x + u^T R u) dt
$$

where $q_1$, $q_2$, $q_3$, $q_4$, and $r$ are non-negative weights 

$$
\mathbf{Q} = \begin{bmatrix} q_1 & 0 & 0 & 0 \\ 0 & q_2 & 0 & 0 \\ 0 & 0 & q_3 & 0 \\ 0 & 0 & 0 & q_4 \end{bmatrix}

\mathbf{R} = [r]
$$

The above cost function is quadratic, and we can therefore a solution is provided by the [linear-quadratic regular (LQR)](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator). 

Specifically, the control law that minimizes the cost function is:

$$
K = - R^{-1} (B^TP) \tilde{x}
$$

Where P is found by solving the [algebraic Riccati equation](https://en.wikipedia.org/wiki/Algebraic_Riccati_equation).

$$
\dot{P}(t) = P A + A^T P + Q - PBR^{-1} B^T P 
$$

Because this problem has infinite time horizons, $\dot{P} = 0$. $P$ therefore is a constant matrix that satisfies:

$$
P A + A^TP + Q - PBR^{-1} B^T P = 0
$$

The evolution of the optimal state vector $\tilde{x}$ can then be described. 
$$
\dot{\tilde{x}} = (A - B R^{-1}B^TP)\tilde{x}
$$


In [112]:
def get_matrices_Q_R(q1, q2, q3, q4, r):
    """
    Returns cost matrices Q and R that effectively place a weight on 
    the cost of errors in the state vector (q1, q2, q3, q4) and the cost
    of non-zero control impulse (r). 

    Parameters:
    q1 (float): relative weight of position of cart.
    q2 (float): relative weight of velocity of cart. 
    q3 (float): relative weight of displacement theta of pole.
    q4 (float): relative weight of angular velocity of pole. 
    r (float): relative weight of control force. 

    Returns:
    R (numpy.ndarray): weights of control vars.
    Q (numpy.ndarray): weights of system vars. 
    """

    R = np.array([r])

    Q = np.array([
        [q1, 0, 0, 0],
        [0, q2, 0, 0],
        [0, 0, q3, 0],
        [0, 0, 0, q4]
    ])

    return Q, R


def find_K_using_lqr(A, B, Q, R):
    """
    Return the matrix P, which satisfies the Riccati differential equation.

    Parameters:
    -----------
    A (numpy.ndarray): System matrix (4,4).
    B (numpy.ndarray): Input matrix (4,1).
    Q (numpy.ndarray): Weights of system vars (4, 4).
    R (numpy.ndarray): Weights of control vars (1, 1). 

    Returns
    -------
    K           : the matrix solution of the Riccati equation. 
    """
    
    P = scipy.linalg.solve_continuous_are(A, B, Q, R)

    return -1/R * B.T @ P